#### EVALUATION FILE

##### This file is to evaluate the loaded models with an specific validation keys

In [1]:
import torch
import os
import numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOCAL_SLP_DATASET_PATH = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(__file__)))),'SLP/danaLab')
SERVER_SLP_DATASET_PATH = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(__file__)))))),'mnt/DADES2/SLP/SLP/danaLab')

List of the valuation patients:

['00082', '00083', '00084', '00085', '00086', '00087', '00088', '00089', '00090', '00091', '00092', '00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00101', '00102']

In [ ]:
# Function to get the numpy arrays of an specific image

def select_file(path,patient,cover,img):

    if os.path.isdir(os.path.join(path,patient)):
        print('Patient:', patient)
        patient_path = os.path.join(path,patient)
        pm_np_path = os.path.join(patient_path,'PMarray')
        ir_np_path = os.path.join(patient_path,'IRraw')

        if os.path.isdir(pm_np_path) and os.path.isdir(ir_np_path):
            pm_cover_path = os.path.join(pm_np_path,cover)
            print('PM cover path:',pm_cover_path)
            ir_cover_path = os.path.join(ir_np_path,cover)
            print('IR cover path:',ir_cover_path)

            if os.path.isdir(pm_cover_path) and os.path.isdir(ir_cover_path):
                pm_file_path = os.path.join(pm_cover_path,img)
                ir_file_path = os.path.join(ir_cover_path,img)
            
                pm_np = np.load(pm_file_path)
                ir_np = np.load(ir_file_path)

                return pm_np,ir_np

In [ ]:
# Function to get all the files of an especific cover

patients = ['00082', '00083', '00084', '00085', '00086', '00087', '00088', '00089',
            '00090', '00091', '00092', '00093', '00094', '00095', '00096', '00097',
            '00098', '00099', '00100', '00101', '00102']

covers = ['cover1','cover2','uncover']

def cover_files(path,patients,cover):
    pm_total_files = []
    ir_total_files = []
    
    for patient in patients:
        if os.path.isdir(os.path.join(path,patient)):
            print('Patient:', patient)
            patient_path = os.path.join(path,patient)
            pm_np_path = os.path.join(patient_path,'PMarray')
            ir_np_path = os.path.join(patient_path,'IRraw')

        if os.path.isdir(pm_np_path) and os.path.isdir(ir_np_path):
            pm_cover_path = os.path.join(pm_np_path,cover)
            ir_cover_path = os.path.join(ir_np_path,cover)
            filenames_pm = os.listdir(pm_cover_path)
            filenames_ir = os.listdir(ir_cover_path)

            for pm_file,ir_file in zip(filenames_pm,filenames_ir):
                pm_total_files.append(np.load(os.path.join(pm_cover_path,pm_file)))
                ir_total_files.append(np.load(os.path.join(ir_cover_path,ir_file)))

    return pm_total_files,ir_total_files

In [ ]:
# Function to get all the files of a patient

def patient_files(path,patient):
    pm_total_files = []
    ir_total_files = []
    
    if os.path.isdir(os.path.join(path,patient)):
        print('Patient:', patient)
        patient_path = os.path.join(path,patient)
        pm_np_path = os.path.join(patient_path,'PMarray')
        ir_np_path = os.path.join(patient_path,'IRraw')

        for category_pm,category_ir in zip(os.listdir(pm_np_path),os.listdir(ir_np_path)):
            pm_cover_path = os.path.join(pm_np_path,category_pm)
            ir_cover_path = os.path.join(ir_np_path,category_ir)
            filenames_pm = os.listdir(pm_cover_path)
            filenames_ir = os.listdir(ir_cover_path)

            for pm_file,ir_file in zip(filenames_pm,filenames_ir):
                pm_total_files.append(np.load(os.path.join(pm_cover_path,pm_file)))
                ir_total_files.append(np.load(os.path.join(ir_cover_path,ir_file)))

    return pm_total_files,ir_total_files


In [ ]:
# Function to get the files that we want and do the transforms and create a dataloader to pass the model

def create_dataloader():
    pass


In [2]:
# Function to evaluate with a model loaded the files that we want with the metrics

def evaluation(model,model_file,metric,files):
    model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
    model.to(DEVICE)

    model.eval()

    with torch.no_grad():

        for input_img, target_img in val_loader:
            break

        input_img = input_img.to(DEVICE)
        target_img = target_img.to(DEVICE)

        output_img = model(input_img)
        loss = criterion(output_img[0], target_img[0])

        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        axes[0].imshow(target_img[0].squeeze().cpu().numpy())
        axes[0].set_title('Target Image')

        axes[1].imshow(output_img[0].squeeze().cpu().numpy())
        axes[1].set_title('Output Image')

        fig.suptitle('Comparison of Target and Output Images', fontsize=12)
        fig.text(0.5, 0.01, f'Loss: {loss.item():.4f}', ha='center')
        plt.savefig(os.path.join(IMG_PATH,'Comparing_output_MSE.png'))
        plt.show()
